# Implementation of kNN algorithme
## 2.1.1 准备：使用python导入数据

In [ ]:
# numpy是科学计算包， 用于python中的各种矩阵运算
import numpy as np
# operator模块是python中内置的运算符模块，
# 它定义了一些算术和比较内置操作的函数。
# operator模块是用c实现的，所以执行速度比python代码快。
import operator 
%matplotlib inline

In [12]:
def createDataSet():
    group = np.array([(1.0, 1.1), (1.0, 1.0), (0, 0), (0, 0.1)])
    labels = ['A', 'A', 'B', 'B']
    return group, labels

In [13]:
group, labels = createDataSet()

In [15]:
group

array([[ 1. ,  1.1],
       [ 1. ,  1. ],
       [ 0. ,  0. ],
       [ 0. ,  0.1]])

In [16]:
labels

['A', 'A', 'B', 'B']

## 2.1.2 实施kNN分类算法
### k 近邻算法的代码如下

In [46]:
# inX: 用于分类的输入向量
# dataSet: 输入的训练样本集
# labels: 输入的训练样本集的标签， 也是一个向量
# k：用于选择最近邻居的数目
# return : inX在该算法下属于的类别

def classify0(inX, dataSet, labels, k):
    dataSetSize = dataSet.shape[0] # 取得dataSetSize的行数
    diffMat = tile(inX, (dataSetSize, 1)) - dataSet # tile 是将 index 在行方向上重复dataSetSize次， 在列方向上重复1次
    sqDiffMat = diffMat**2
    sqDistances = sqDiffMat.sum(axis=1) # sum(axis=1) 是让矩阵的每一行向量相加 (X11-X1)^2 + (X10-x0)^2
    distances = sqDistances**0.5 # 对距离的平方开根号
    sortedDistIndicies = distances.argsort() #argsort对数组进行排序（从小到大）， 返回的是indice
    classCount = {}
    # 选择距离最小的k个点
    for i in range(3):
        voteIlabel = labels[sortedDistIndicies[i]]
        classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1
    # sorted 函数是排序函数， 第一个参数是排序对象，可以是list或者是iterable(这里是第二个)，
    # 第二个参数是比较的key， 这里是dictionnary里面的元素的值
    # 第三个参数reverse表示逆序排序
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]
    
print("The class of inX is ",classify0([0,0], group, labels, 3))


('The class of inX is ', 'B')


In [27]:
dataSet = group
dataSet

array([[ 1. ,  1.1],
       [ 1. ,  1. ],
       [ 0. ,  0. ],
       [ 0. ,  0.1]])

In [24]:
dataSetSize = dataSet.shape[0] # 取得dataSetSize的行数
dataSetSize

4L

In [25]:
inX = [0, 0]

In [29]:
diffMat = tile(inX, (dataSetSize, 1)) - dataSet # tile 是将 index 在行方向上重复dataSetSize次， 在列方向上重复1次
diffMat

array([[-1. , -1.1],
       [-1. , -1. ],
       [ 0. ,  0. ],
       [ 0. , -0.1]])

In [30]:
sqDiffMat = diffMat**2
sqDiffMat

array([[ 1.  ,  1.21],
       [ 1.  ,  1.  ],
       [ 0.  ,  0.  ],
       [ 0.  ,  0.01]])

In [33]:
sqDistances = sqDiffMat.sum(axis=1) # sum(axis=1) 是让矩阵的每一行向量相加 (X11-X1)^2 + (X10-x0)^2
sqDistances

array([ 2.21,  2.  ,  0.  ,  0.01])

In [35]:
distances = sqDistances**0.5
distances 

array([ 1.48660687,  1.41421356,  0.        ,  0.1       ])

In [36]:
sortedDistIndicies = distances.argsort() #argsort对数组进行排序（从小到大）， 返回的是indice
sortedDistIndicies

array([2, 3, 1, 0], dtype=int64)

In [39]:
classCount = {}

In [40]:
# 选择距离最小的k个点
for i in range(3):
    voteIlabel = labels[sortedDistIndicies[i]]
    classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1
voteIlabel
classCount

{'A': 1, 'B': 2}

In [43]:
# sorted 函数是排序函数， 第一个参数是排序对象，可以是list或者是iterable(这里是第二个)，
# 第二个参数是比较的key， 这里是dictionnary里面的元素的值
# 第三个参数reverse表示逆序排序
sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
sortedClassCount

[('B', 2), ('A', 1)]

## 2.2示例：使用k近邻算法改进约会网站的配对结果
### 2.2.1 准备数据：从文本文件中解析数据

In [56]:
# file2matrix 用来处理输入格式问题
# 输入：文件名字符串
# 输出: 训练样本矩阵和类标签向量

def file2matrix(filename):
    fr = open(filename) # 得到文件
    arrayOfLines = fr.readlines() # 一次读取整个文件，并将文件内容分析成一个行的列表
    numberOfLines = len(arrayOfLines) # 得到文件的行数
    returnMat = zeros((numberOfLines, 3)) # 创建返回的numpy矩阵， 维度为：numberOfLines x 3
    
    classLabelVector = []
    index = 0
    # 解析文件数据得到列表
    for line in arrayOfLines:
        # strip : This method returns a copy of the string in which 
        # all chars have been stripped from the beginning and the end of the string.
        line = line.strip() 
        # split: The method split() returns a list of all the words in the string, 
        # using str as the separator (splits on all whitespace if left unspecified), 
        # optionally limiting the number of splits to num.
        listFromLine = line.split('\t')
        returnMat[index:] = listFromLine[0:3]
        classLabelVector.append(int(listFromLine[-1])) # 将label加入classLabelVector这个向量中
    return returnMat, classLabelVector

In [57]:
datingDataMat, datingLabels = file2matrix('datingTestSet2.txt')

In [58]:
datingDataMat

array([[  4.37570000e+04,   7.88260100e+00,   1.33244600e+00],
       [  4.37570000e+04,   7.88260100e+00,   1.33244600e+00],
       [  4.37570000e+04,   7.88260100e+00,   1.33244600e+00],
       ..., 
       [  4.37570000e+04,   7.88260100e+00,   1.33244600e+00],
       [  4.37570000e+04,   7.88260100e+00,   1.33244600e+00],
       [  4.37570000e+04,   7.88260100e+00,   1.33244600e+00]])

In [60]:
datingLabels[0:5]

[3, 2, 1, 1, 1]

### 2.2.2 分析数据： 使用Matpltlib创建散点图

In [ ]:
# 导入绘图包
import matplotlib
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111) # add_subplot(ijk) means i x j grids kth subplot
ax.scatter(datingDataMat[:,1], datingDataMat[:,2])
plt.show()